In [1]:
# Effort to reproduce results of
# https://www.biorxiv.org/content/10.1101/2020.01.08.898288v1.full#ref-21
# "Individual differences among deep neural network models"


In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, Dropout, GlobalAveragePooling2D,
                                    MaxPooling2D, Activation, Dense, Layer)
from tensorflow.keras.datasets import cifar10
from tensorflow.compat.v2.nn import local_response_normalization as lrn
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# num_validation_samples = 15000
# x_validation = x_train[:num_validation_samples].astype('float32')/255
# y_validation = y_train[:num_validation_samples]
# x_train = x_train[num_validation_samples:].astype('float32')/255
# y_train = y_train[num_validation_samples:]
# x_test = x_test.astype('float32')/255

num_validation_samples = 15000
# x_test = x_test.astype('float32')/255


print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
# print(x_validation.shape[0], 'validation samples')
print(x_test.shape[0], 'test samples')

num_classes = 10
# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
#y_validation = to_categorical(y_validation, num_classes)
y_test = to_categorical(y_test, num_classes)

class LRN(Layer):
    def call(self, inp):
        return lrn(inp)


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# All_CNN_C architecture
from tensorflow.keras.layers import BatchNormalization
def init_model(architecture: str, seed: int):
    
    if (architecture == 'all_cnn_c'):
        model = Sequential()
        # model.add(Dropout(0.2, input_shape=x_train.shape[1:])) #input shape from keras cifar10 example
        model.add(Conv2D(96, (3, 3), input_shape=x_train.shape[1:], padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), 
                         bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), strides=2, padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), strides=2, padding='same',kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(10, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(GlobalAveragePooling2D())
        model.add(Activation('softmax'))
        
        model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, clipnorm=500),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    
    elif (architecture =='vgg753'):
        model = Sequential()
        model.add(Conv2D(96, (7, 7), strides=2, input_shape=x_train.shape[1:], kernel_regularizer=l2(1e-5), 
                         bias_regularizer=l2(1e-5), bias_initializer='zeros', kernel_initializer=he_normal(seed)))
#         # A little unsure about the order of these two
#         model.add(LRN())
#         model.add(Activation('relu'))
#         ##

        # Replace LRN with batch
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
        model.add(Conv2D(128, (5, 5), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
                         bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
                         bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
        model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
                         bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
        model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
                         bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
        model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
        model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
                         bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
        model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
                         bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
        model.add(GlobalAveragePooling2D())
        model.add(Dense(10, activation='softmax'))

        model.compile(optimizer=Adam(learning_rate=0.01, epsilon=0.1, clipnorm=500),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

    else:
        raise

    return model

'''
USE TENSORFLOW INSTEAD OF KERAS FOR DATA GEN
trainData = tf.data.Dataset.from_tensor_slices((x_train, y_train))
To run just a single gpu: set CUDA_VISIBLE_DEVICES 1 in environment var
'''

'\nUSE TENSORFLOW INSTEAD OF KERAS FOR DATA GEN\ntrainData = tf.data.Dataset.from_tensor_slices((x_train, y_train))\nTo run just a single gpu: set CUDA_VISIBLE_DEVICES 1 in environment var\n'

In [2]:
# Load datasets

import correlations
x_predict, y_predict = correlations.make_test_data()

x_train_new = np.load('x_train_new.npy')
y_train_new = np.load('y_train_new.npy')
x_test_new = np.load('x_test_new.npy')
y_test_new = np.load('y_test_new.npy')



# K.clear_session()
# # Unsure if samplewise_std_normalization is equivalent to GCN
# train_datagen = ImageDataGenerator(zca_whitening=True)
# test_datagen = ImageDataGenerator(zca_whitening=True)
# train_datagen.fit(x_train)
# test_datagen.fit(x_test)
# train_generator = train_datagen.flow(
#     x_train,
#     y_train,
#     batch_size=50000,
#     # Maybe stick something here for seed
# )
# x_train_new = train_generator[0][0]
# y_train_new = train_generator[0][1]
# test_generator = test_datagen.flow(
#     x_test,
#     y_test,
#     batch_size=10000
# )
# x_test_new = test_generator[0][0]
# y_test_new = test_generator[0][1]
# np.save('x_train_new.npy', x_train_new)
# np.save('y_train_new.npy', y_train_new)
# np.save('x_test_new.npy', x_test_new)
# np.save('y_test_new.npy', y_test_new)

Image 0
Counts:  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Image 1
Counts:  [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
Image 2
Counts:  [0, 0, 0, 1, 0, 0, 0, 0, 2, 0]
Image 3
Counts:  [1, 0, 0, 1, 0, 0, 0, 0, 2, 0]
Image 4
Counts:  [1, 0, 0, 1, 0, 0, 1, 0, 2, 0]
Image 5
Counts:  [1, 0, 0, 1, 0, 0, 2, 0, 2, 0]
Image 6
Counts:  [1, 1, 0, 1, 0, 0, 2, 0, 2, 0]
Image 7
Counts:  [1, 1, 0, 1, 0, 0, 3, 0, 2, 0]
Image 8
Counts:  [1, 1, 0, 2, 0, 0, 3, 0, 2, 0]
Image 9
Counts:  [1, 2, 0, 2, 0, 0, 3, 0, 2, 0]
Image 10
Counts:  [2, 2, 0, 2, 0, 0, 3, 0, 2, 0]
Image 11
Counts:  [2, 2, 0, 2, 0, 0, 3, 0, 2, 1]
Image 12
Counts:  [2, 2, 0, 2, 0, 1, 3, 0, 2, 1]
Image 13
Counts:  [2, 2, 0, 2, 0, 1, 3, 1, 2, 1]
Image 14
Counts:  [2, 2, 0, 2, 0, 1, 3, 1, 2, 2]
Image 15
Counts:  [2, 2, 0, 2, 0, 1, 3, 1, 3, 2]
Image 16
Counts:  [2, 2, 0, 2, 0, 2, 3, 1, 3, 2]
Image 17
Counts:  [2, 2, 0, 2, 0, 2, 3, 2, 3, 2]
Image 18
Counts:  [2, 2, 0, 2, 0, 2, 3, 2, 4, 2]
Image 19
Counts:  [2, 2, 0, 2, 0, 2, 4, 2, 4, 2]
Image 20
Counts:  [2, 2, 0, 2,

Counts:  [69, 51, 62, 60, 54, 67, 76, 62, 69, 76]
Image 646
Counts:  [69, 51, 62, 61, 54, 67, 76, 62, 69, 76]
Image 647
Counts:  [69, 51, 62, 61, 55, 67, 76, 62, 69, 76]
Image 648
Counts:  [69, 51, 62, 61, 56, 67, 76, 62, 69, 76]
Image 649
Counts:  [69, 51, 62, 61, 56, 68, 76, 62, 69, 76]
Image 650
Counts:  [69, 51, 62, 62, 56, 68, 76, 62, 69, 76]
Image 651
Counts:  [69, 51, 62, 62, 56, 68, 76, 62, 69, 77]
Image 652
Counts:  [69, 51, 62, 62, 56, 69, 76, 62, 69, 77]
Image 653
Counts:  [69, 51, 62, 62, 56, 69, 77, 62, 69, 77]
Image 654
Counts:  [69, 51, 62, 62, 56, 69, 77, 62, 69, 78]
Image 655
Counts:  [69, 51, 63, 62, 56, 69, 77, 62, 69, 78]
Image 656
Counts:  [69, 52, 63, 62, 56, 69, 77, 62, 69, 78]
Image 657
Counts:  [69, 53, 63, 62, 56, 69, 77, 62, 69, 78]
Image 658
Counts:  [69, 53, 63, 62, 57, 69, 77, 62, 69, 78]
Image 659
Counts:  [69, 54, 63, 62, 57, 69, 77, 62, 69, 78]
Image 660
Counts:  [69, 54, 63, 62, 57, 69, 77, 62, 69, 79]
Image 661
Counts:  [69, 54, 63, 62, 58, 69, 77, 62

In [6]:
# Get callback to perform RSA and get reps during training for trajectory work
from tensorflow.keras.callbacks import Callback
import correlations

class Trajectory_Callback(Callback):
    '''
    Pre: Must define instance_num, layer_arr, x_predict
    '''
    def on_epoch_end(self, epoch, logs=None):
        if epoch in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                     49, 99, 149, 199, 249, 299, 349]:
            # It's really nice how I can update instance_num outside of the def of this and it updates inside            
            print('\n\nSnapshot instance', str(instance_num+1), 'at epoch', str(int(epoch)+1))
            acts = correlations.get_acts(self.model, layer_arr, x_predict)
            np.save('RDMs_and_Acts/Acts/Instance'+str(instance_num+1)+'_Epoch'+str(int(epoch)+1)+'.npy', acts)
            print('\n')

In [ ]:
# Note: There are additional imports needed if you want VGG753
import numpy as np
import tensorflow as tf
import os
import random
import correlations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, Dropout, GlobalAveragePooling2D,
                                    MaxPooling2D, Activation, Dense, Layer)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import LearningRateScheduler, Callback
from tensorflow.keras import backend as K 

# Set seed value for python and np
seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

# Load predict datasets and whitened datasets
x_predict, y_predict = correlations.make_test_data()
x_train_new = np.load('x_train_new.npy')
y_train_new = np.load('y_train_new.npy')
x_test_new = np.load('x_test_new.npy')
y_test_new = np.load('y_test_new.npy')

# Function to make models
def init_model(architecture: str, seed: int):
    
    if (architecture == 'all_cnn_c'):
        model = Sequential()
        # model.add(Dropout(0.2, input_shape=x_train.shape[1:])) #input shape from keras cifar10 example
        model.add(Conv2D(96, (3, 3), input_shape=x_train_new.shape[1:], padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), 
                         bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(96, (3, 3), strides=2, padding='same', bias_regularizer=l2(1e-5),
                         kernel_regularizer=l2(1e-5), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (3, 3), strides=2, padding='same',kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        # model.add(Dropout(0.5))
        model.add(Conv2D(192, (3, 3), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5),
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(192, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(Conv2D(10, (1, 1), padding='valid', kernel_regularizer=l2(1e-5), bias_regularizer=l2(1e-5), 
                         kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
        model.add(GlobalAveragePooling2D())
        model.add(Activation('softmax'))
        
        model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, clipnorm=500),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    
#     elif (architecture =='vgg753'):
#         model = Sequential()
#         model.add(Conv2D(96, (7, 7), strides=2, input_shape=x_train.shape[1:], kernel_regularizer=l2(1e-5), 
#                          bias_regularizer=l2(1e-5), bias_initializer='zeros', kernel_initializer=he_normal(seed)))
#         model.add(BatchNormalization())
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(128, (5, 5), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(2, 2)))
#         model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(Conv2D(1024, (3, 3), padding='same', kernel_regularizer=l2(1e-5), bias_initializer='zeros', 
#                          bias_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), activation='relu'))
#         model.add(GlobalAveragePooling2D())
#         model.add(Dense(10, activation='softmax'))

#         model.compile(optimizer=Adam(learning_rate=0.01, epsilon=0.1, clipnorm=500),
#                       loss='categorical_crossentropy',
#                       metrics=['accuracy'])

    else:
        raise

    return model

# Scheduler callback
def scheduler(epoch, lr):
    if (epoch == 200 or epoch == 250 or epoch == 300):
        return lr * 0.1
    return lr

lr_callback = LearningRateScheduler(scheduler)

# Trajectory callback
class Trajectory_Callback(Callback):
    '''
    Pre: Must define instance_num, layer_arr, x_predict
    '''
    def on_epoch_end(self, epoch, logs=None):
        if epoch in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                     49, 99, 149, 199, 249, 299, 349]:
            # It's really nice how I can update instance_num outside of the def of this and it updates inside            
            print('\n\nSnapshot instance', str(instance_num+1), 'at epoch', str(int(epoch)+1))
            acts = correlations.get_acts(self.model, layer_arr, x_predict)
            np.save('RDMs_and_Acts/Acts/Instance'+str(instance_num+1)+'_Epoch'+str(int(epoch)+1)+'.npy', acts)
            print('\n')
            
# Training loop
num_instances = 10
for i in range(num_instances):
    
    # TF seed needs to be reset after every clear_session()
    K.clear_session()
    tf.random.set_seed(seed_value)

    # All_CNN_C
    print('Training All_CNN_C with seed', str(i))
    all_cnn_c = init_model('all_cnn_c', seed=i)

    instance_num = i
    layer_arr = [8]
    
    a_history = all_cnn_c.fit(
                x_train_new,
                y_train_new,
                batch_size=128,
                epochs=350,
                callbacks=[lr_callback, Trajectory_Callback()],
                validation_data=(x_test_new, y_test_new),
                shuffle=False)
    
    all_cnn_c.save('Models/Primary/Version_2/all_cnn_c_instance_'+str(i)+'.h5')

Image 0
Counts:  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Image 1
Counts:  [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
Image 2
Counts:  [0, 0, 0, 1, 0, 0, 0, 0, 2, 0]
Image 3
Counts:  [1, 0, 0, 1, 0, 0, 0, 0, 2, 0]
Image 4
Counts:  [1, 0, 0, 1, 0, 0, 1, 0, 2, 0]
Image 5
Counts:  [1, 0, 0, 1, 0, 0, 2, 0, 2, 0]
Image 6
Counts:  [1, 1, 0, 1, 0, 0, 2, 0, 2, 0]
Image 7
Counts:  [1, 1, 0, 1, 0, 0, 3, 0, 2, 0]
Image 8
Counts:  [1, 1, 0, 2, 0, 0, 3, 0, 2, 0]
Image 9
Counts:  [1, 2, 0, 2, 0, 0, 3, 0, 2, 0]
Image 10
Counts:  [2, 2, 0, 2, 0, 0, 3, 0, 2, 0]
Image 11
Counts:  [2, 2, 0, 2, 0, 0, 3, 0, 2, 1]
Image 12
Counts:  [2, 2, 0, 2, 0, 1, 3, 0, 2, 1]
Image 13
Counts:  [2, 2, 0, 2, 0, 1, 3, 1, 2, 1]
Image 14
Counts:  [2, 2, 0, 2, 0, 1, 3, 1, 2, 2]
Image 15
Counts:  [2, 2, 0, 2, 0, 1, 3, 1, 3, 2]
Image 16
Counts:  [2, 2, 0, 2, 0, 2, 3, 1, 3, 2]
Image 17
Counts:  [2, 2, 0, 2, 0, 2, 3, 2, 3, 2]
Image 18
Counts:  [2, 2, 0, 2, 0, 2, 3, 2, 4, 2]
Image 19
Counts:  [2, 2, 0, 2, 0, 2, 4, 2, 4, 2]
Image 20
Counts:  [2, 2, 0, 2,

Counts:  [73, 56, 68, 70, 60, 71, 83, 66, 73, 83]
Image 703
Counts:  [73, 56, 68, 70, 60, 71, 83, 66, 74, 83]
Image 704
Counts:  [73, 56, 68, 70, 60, 71, 83, 67, 74, 83]
Image 705
Counts:  [73, 56, 68, 70, 61, 71, 83, 67, 74, 83]
Image 706
Counts:  [73, 56, 69, 70, 61, 71, 83, 67, 74, 83]
Image 707
Counts:  [73, 56, 69, 70, 61, 71, 83, 67, 74, 84]
Image 708
Counts:  [73, 56, 69, 70, 61, 71, 83, 67, 75, 84]
Image 709
Counts:  [73, 56, 69, 70, 61, 71, 83, 67, 76, 84]
Image 710
Counts:  [73, 56, 69, 70, 61, 71, 84, 67, 76, 84]
Image 711
Counts:  [73, 56, 69, 70, 61, 71, 84, 67, 77, 84]
Image 712
Counts:  [73, 56, 69, 70, 61, 71, 84, 67, 78, 84]
Image 713
Counts:  [73, 56, 69, 70, 61, 71, 84, 68, 78, 84]
Image 714
Counts:  [73, 56, 69, 70, 62, 71, 84, 68, 78, 84]
Image 715
Counts:  [73, 56, 69, 71, 62, 71, 84, 68, 78, 84]
Image 716
Counts:  [73, 56, 69, 72, 62, 71, 84, 68, 78, 84]
Image 717
Counts:  [73, 56, 69, 72, 62, 71, 84, 68, 79, 84]
Image 718
Counts:  [73, 56, 69, 72, 63, 71, 84, 68

Training All_CNN_C with seed 0
Train on 50000 samples, validate on 10000 samples
Epoch 1/350
49920/50000 [============================>.] - ETA: 0s - loss: 2.3237 - accuracy: 0.0958

Snapshot instance 1 at epoch 1
Layer 9
Getting activations...


50000/50000 [==============================] - 183s 4ms/sample - loss: 2.3237 - accuracy: 0.0957 - val_loss: 2.3091 - val_accuracy: 0.1032
Epoch 2/350
49920/50000 [============================>.] - ETA: 0s - loss: 2.2931 - accuracy: 0.1319

Snapshot instance 1 at epoch 2
Layer 9
Getting activations...


50000/50000 [==============================] - 182s 4ms/sample - loss: 2.2932 - accuracy: 0.1317 - val_loss: 2.2963 - val_accuracy: 0.0981
Epoch 3/350
49920/50000 [============================>.] - ETA: 0s - loss: 2.2810 - accuracy: 0.1264

Snapshot instance 1 at epoch 3
Layer 9
Getting activations...


50000/50000 [==============================] - 182s 4ms/sample - loss: 2.2810 - accuracy: 0.1263 - val_loss: 2.2687 - val_accuracy: 0.1147
Epoc

## Everything below this cell is testing stuff

In [6]:
x_train_new

array([[[[ 3.59662890e-01,  4.23392653e-01, -5.32661915e-01],
         [-5.56859970e-02,  3.04873168e-01,  3.39735091e-01],
         [ 3.05605084e-01,  4.43103731e-01,  1.90269411e-01],
         ...,
         [-7.59185851e-01, -2.29820177e-01, -8.20753098e-01],
         [-1.34929791e-01, -7.22570866e-02, -9.76636037e-02],
         [ 6.00084245e-01,  9.18413997e-02, -1.10491514e+00]],

        [[ 6.98050737e-01,  7.90449798e-01,  8.48339617e-01],
         [ 1.43036520e+00,  1.81177795e-01,  4.29874271e-01],
         [ 3.56751621e-01,  1.07323861e+00,  7.28286028e-01],
         ...,
         [ 4.98441815e-01,  2.45685562e-01,  4.00865823e-01],
         [ 5.78554273e-01, -9.14298519e-02, -2.09718421e-02],
         [ 7.88109452e-02,  2.12441683e-01,  3.19318324e-02]],

        [[-2.78410256e-01, -1.97635502e-01, -1.33945227e+00],
         [-1.00501323e+00, -3.10764492e-01,  2.63749659e-02],
         [ 4.68385905e-01, -5.48063278e-01,  1.03883475e-01],
         ...,
         [ 7.20174909e-0

In [5]:
# Test correlations.py
import correlations
from tensorflow.keras.models import load_model
model = load_model('Models/Primary/Version_1/all_cnn_c_0.h5')
layer_arr = [0, 1, 2, 3, 4, 5, 6, 7, 8]
x_predict, y_predict = correlations.make_test_data()

RDMs = correlations.make_RDMs(model, layer_arr, x_predict)
acts = correlations.get_acts(model, layer_arr, x_predict)

Image 0
Counts:  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Image 1
Counts:  [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
Image 2
Counts:  [0, 0, 0, 1, 0, 0, 0, 0, 2, 0]
Image 3
Counts:  [1, 0, 0, 1, 0, 0, 0, 0, 2, 0]
Image 4
Counts:  [1, 0, 0, 1, 0, 0, 1, 0, 2, 0]
Image 5
Counts:  [1, 0, 0, 1, 0, 0, 2, 0, 2, 0]
Image 6
Counts:  [1, 1, 0, 1, 0, 0, 2, 0, 2, 0]
Image 7
Counts:  [1, 1, 0, 1, 0, 0, 3, 0, 2, 0]
Image 8
Counts:  [1, 1, 0, 2, 0, 0, 3, 0, 2, 0]
Image 9
Counts:  [1, 2, 0, 2, 0, 0, 3, 0, 2, 0]
Image 10
Counts:  [2, 2, 0, 2, 0, 0, 3, 0, 2, 0]
Image 11
Counts:  [2, 2, 0, 2, 0, 0, 3, 0, 2, 1]
Image 12
Counts:  [2, 2, 0, 2, 0, 1, 3, 0, 2, 1]
Image 13
Counts:  [2, 2, 0, 2, 0, 1, 3, 1, 2, 1]
Image 14
Counts:  [2, 2, 0, 2, 0, 1, 3, 1, 2, 2]
Image 15
Counts:  [2, 2, 0, 2, 0, 1, 3, 1, 3, 2]
Image 16
Counts:  [2, 2, 0, 2, 0, 2, 3, 1, 3, 2]
Image 17
Counts:  [2, 2, 0, 2, 0, 2, 3, 2, 3, 2]
Image 18
Counts:  [2, 2, 0, 2, 0, 2, 3, 2, 4, 2]
Image 19
Counts:  [2, 2, 0, 2, 0, 2, 4, 2, 4, 2]
Image 20
Counts:  [2, 2, 0, 2,

Counts:  [73, 56, 69, 72, 64, 71, 84, 68, 81, 85]
Image 723
Counts:  [73, 57, 69, 72, 64, 71, 84, 68, 81, 85]
Image 724
Counts:  [73, 57, 69, 72, 64, 71, 84, 68, 82, 85]
Image 725
Counts:  [73, 57, 70, 72, 64, 71, 84, 68, 82, 85]
Image 726
Counts:  [73, 58, 70, 72, 64, 71, 84, 68, 82, 85]
Image 727
Counts:  [73, 58, 70, 73, 64, 71, 84, 68, 82, 85]
Image 728
Counts:  [73, 58, 70, 73, 64, 71, 85, 68, 82, 85]
Image 729
Counts:  [73, 58, 70, 73, 64, 72, 85, 68, 82, 85]
Image 730
Counts:  [73, 58, 70, 73, 65, 72, 85, 68, 82, 85]
Image 731
Counts:  [73, 58, 71, 73, 65, 72, 85, 68, 82, 85]
Image 732
Counts:  [73, 58, 71, 73, 65, 72, 85, 69, 82, 85]
Image 733
Counts:  [73, 58, 71, 73, 65, 72, 85, 69, 82, 86]
Image 734
Counts:  [73, 58, 71, 73, 65, 72, 85, 69, 82, 87]
Image 735
Counts:  [73, 58, 71, 73, 66, 72, 85, 69, 82, 87]
Image 736
Counts:  [73, 59, 71, 73, 66, 72, 85, 69, 82, 87]
Image 737
Counts:  [73, 59, 71, 73, 67, 72, 85, 69, 82, 87]
Image 738
Counts:  [73, 60, 71, 73, 67, 72, 85, 69

(98304, 1000)
Getting RDM...
Layer 2
Getting representation...
(98304, 1000)
Getting RDM...
Layer 3
Getting representation...
(24576, 1000)
Getting RDM...
Layer 4
Getting representation...
(49152, 1000)
Getting RDM...
Layer 5
Getting representation...
(49152, 1000)
Getting RDM...
Layer 6
Getting representation...
(12288, 1000)
Getting RDM...
Layer 7
Getting representation...
(12288, 1000)
Getting RDM...
Layer 8
Getting representation...
(12288, 1000)
Getting RDM...
Layer 9
Getting representation...
(640, 1000)
Getting RDM...
Layer 1
Getting activations...
Layer 2
Getting activations...
Layer 3
Getting activations...
Layer 4
Getting activations...
Layer 5
Getting activations...
Layer 6
Getting activations...
Layer 7
Getting activations...
Layer 8
Getting activations...
Layer 9
Getting activations...


In [38]:
K.clear_session()
mdl = init_model('all_cnn_c', seed=999)
print(mdl.get_weights())

[array([[[[-0.50432205, -0.07790265, -0.02935393, ...,  0.30354655,
          -0.37062874,  0.26897284],
         [ 0.15036999, -0.35827142,  0.01860022, ...,  0.0376279 ,
           0.01976478, -0.2421805 ],
         [-0.5239501 , -0.08314488,  0.16135833, ..., -0.4626265 ,
          -0.23466884,  0.6086181 ]],

        [[ 0.03842236, -0.02121304, -0.03650312, ..., -0.12125434,
           0.03335589,  0.21451873],
         [-0.04562262, -0.57643175,  0.08655257, ...,  0.2626094 ,
          -0.46007645, -0.2894683 ],
         [-0.22372884, -0.09718585, -0.15044308, ..., -0.2930702 ,
          -0.2533019 ,  0.2070803 ]],

        [[-0.09400132, -0.16578957,  0.08565225, ..., -0.12310454,
           0.5509518 , -0.1941309 ],
         [-0.27058023, -0.08447935,  0.57237154, ...,  0.05362112,
          -0.29129258,  0.17526712],
         [-0.03541306, -0.33324245, -0.19152407, ...,  0.29984906,
           0.17666754, -0.05128127]]],


       [[[ 0.49535248,  0.24324092, -0.22501266, ...,  

In [40]:
x_train_new

array([[[[ 4.98730958e-01, -1.49664652e+00, -8.61457527e-01],
         [-7.55373538e-01,  9.21619534e-01,  2.83206217e-02],
         [-1.88578069e+00,  1.52964687e+00, -5.34050822e-01],
         ...,
         [-2.47838998e+00, -1.75947750e+00, -1.52870154e+00],
         [ 9.13508892e-01,  2.12430835e-01,  1.51381350e+00],
         [ 2.76861429e+00, -9.08216298e-01, -8.54953110e-01]],

        [[ 4.68991458e-01,  2.38613367e-01, -2.69154131e-01],
         [-6.65015996e-01, -4.48480695e-01, -8.26985061e-01],
         [-3.14204335e-01,  4.90182370e-01,  8.27626169e-01],
         ...,
         [ 2.18287021e-01, -2.24888539e+00,  8.92874077e-02],
         [-6.47441924e-01, -9.59946990e-01,  4.27951962e-01],
         [-3.64481866e-01,  2.15834594e+00, -4.89134490e-01]],

        [[ 1.55287832e-01, -9.25809264e-01,  3.51847589e-01],
         [-8.66813302e-01, -2.74568081e-01, -6.26631439e-01],
         [ 1.56941950e-01, -2.12088153e-01,  1.08123660e-01],
         ...,
         [ 1.09477866e+0

In [14]:
x_predict, y_predict = correlations.make_test_data()

Image 0
Counts:  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Image 1
Counts:  [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
Image 2
Counts:  [0, 0, 0, 1, 0, 0, 0, 0, 2, 0]
Image 3
Counts:  [1, 0, 0, 1, 0, 0, 0, 0, 2, 0]
Image 4
Counts:  [1, 0, 0, 1, 0, 0, 1, 0, 2, 0]
Image 5
Counts:  [1, 0, 0, 1, 0, 0, 2, 0, 2, 0]
Image 6
Counts:  [1, 1, 0, 1, 0, 0, 2, 0, 2, 0]
Image 7
Counts:  [1, 1, 0, 1, 0, 0, 3, 0, 2, 0]
Image 8
Counts:  [1, 1, 0, 2, 0, 0, 3, 0, 2, 0]
Image 9
Counts:  [1, 2, 0, 2, 0, 0, 3, 0, 2, 0]
Image 10
Counts:  [2, 2, 0, 2, 0, 0, 3, 0, 2, 0]
Image 11
Counts:  [2, 2, 0, 2, 0, 0, 3, 0, 2, 1]
Image 12
Counts:  [2, 2, 0, 2, 0, 1, 3, 0, 2, 1]
Image 13
Counts:  [2, 2, 0, 2, 0, 1, 3, 1, 2, 1]
Image 14
Counts:  [2, 2, 0, 2, 0, 1, 3, 1, 2, 2]
Image 15
Counts:  [2, 2, 0, 2, 0, 1, 3, 1, 3, 2]
Image 16
Counts:  [2, 2, 0, 2, 0, 2, 3, 1, 3, 2]
Image 17
Counts:  [2, 2, 0, 2, 0, 2, 3, 2, 3, 2]
Image 18
Counts:  [2, 2, 0, 2, 0, 2, 3, 2, 4, 2]
Image 19
Counts:  [2, 2, 0, 2, 0, 2, 4, 2, 4, 2]
Image 20
Counts:  [2, 2, 0, 2,

Counts:  [73, 58, 71, 73, 65, 72, 85, 69, 82, 86]
Image 734
Counts:  [73, 58, 71, 73, 65, 72, 85, 69, 82, 87]
Image 735
Counts:  [73, 58, 71, 73, 66, 72, 85, 69, 82, 87]
Image 736
Counts:  [73, 59, 71, 73, 66, 72, 85, 69, 82, 87]
Image 737
Counts:  [73, 59, 71, 73, 67, 72, 85, 69, 82, 87]
Image 738
Counts:  [73, 60, 71, 73, 67, 72, 85, 69, 82, 87]
Image 739
Counts:  [73, 60, 71, 74, 67, 72, 85, 69, 82, 87]
Image 740
Counts:  [73, 60, 72, 74, 67, 72, 85, 69, 82, 87]
Image 741
Counts:  [73, 60, 72, 74, 67, 72, 85, 70, 82, 87]
Image 742
Counts:  [74, 60, 72, 74, 67, 72, 85, 70, 82, 87]
Image 743
Counts:  [74, 60, 72, 74, 67, 72, 85, 71, 82, 87]
Image 744
Counts:  [74, 60, 72, 74, 67, 72, 85, 71, 82, 88]
Image 745
Counts:  [74, 60, 72, 74, 67, 72, 85, 72, 82, 88]
Image 746
Counts:  [74, 60, 72, 74, 67, 72, 86, 72, 82, 88]
Image 747
Counts:  [74, 60, 72, 74, 67, 72, 87, 72, 82, 88]
Image 748
Counts:  [74, 60, 73, 74, 67, 72, 87, 72, 82, 88]
Image 749
Counts:  [74, 60, 73, 74, 67, 73, 87, 72

In [11]:
K.clear_session()

# Seed value (can actually be different for each attribution step)
seed_value= 0

# # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
# import os
# os.environ['PYTHONHASHSEED']=str(seed_value)

# # 2. Set `python` built-in pseudo-random generator at a fixed value
# import random
# random.seed(seed_value)

# # 3. Set `numpy` pseudo-random generator at a fixed value
# import numpy as np
# np.random.seed(seed_value)

# # 4. Set `tensorflow` pseudo-random generator at a fixed value
# import tensorflow as tf
tf.random.set_seed(seed_value)

# All_CNN_C
print('Training All_CNN_C with seed', str(1000))
all_cnn_c = init_model('all_cnn_c', seed=1000)

def scheduler(epoch, lr):
    if (epoch == 200 or epoch == 250 or epoch == 300):
        return lr * 0.1
    return lr
    
lr_callback = LearningRateScheduler(scheduler)
instance_num = 0
layer_arr = [8]
a_history = all_cnn_c.fit(
            x_train_new,
            y_train_new,
            batch_size=128,
            epochs=350,
            callbacks=[lr_callback, Trajectory_Callback()],
            validation_data=(x_test_new, y_test_new),
            shuffle=False)

Training All_CNN_C with seed 1000
Train on 50000 samples, validate on 10000 samples
Epoch 1/350
49920/50000 [============================>.] - ETA: 0s - loss: 2.2818 - accuracy: 0.1190

Snapshot instance 1 at epoch 1
Layer 9
Getting activations...
50000/50000 [==============================] - 183s 4ms/sample - loss: 2.2817 - accuracy: 0.1190 - val_loss: 2.2810 - val_accuracy: 0.1393
Epoch 2/350
49920/50000 [============================>.] - ETA: 0s - loss: 2.1400 - accuracy: 0.1833

Snapshot instance 1 at epoch 2
Layer 9
Getting activations...
50000/50000 [==============================] - 182s 4ms/sample - loss: 2.1400 - accuracy: 0.1832 - val_loss: 2.1377 - val_accuracy: 0.1496
Epoch 3/350
49920/50000 [============================>.] - ETA: 0s - loss: 1.9687 - accuracy: 0.2731

Snapshot instance 1 at epoch 3
Layer 9
Getting activations...
50000/50000 [==============================] - 183s 4ms/sample - loss: 1.9688 - accuracy: 0.2731 - val_loss: 1.9795 - val_accuracy: 0.2559
Epoch 4

KeyboardInterrupt: 

In [19]:
acts = np.load('RDMs_and_Acts/RDMs/Instance0_Epoch1.npy')

In [20]:
acts.shape

(9, 1000, 1000)

In [5]:
# VGG753

print('Training VGG753 with seed', str(1000))
K.clear_session()
vgg753 = init_model('vgg753', seed=2)

def scheduler(epoch, lr):
    if (epoch == 100 or epoch == 150 or epoch == 200):
        return lr * 0.1
    return lr
    
callback = LearningRateScheduler(scheduler)

a_history = vgg753.fit(
            x_train,
            y_train,
            batch_size=512,
            epochs=250,
            callbacks=[callback],
            validation_data=(x_test, y_test),
            shuffle=False)

Training VGG753 with seed 1000
Train on 50000 samples, validate on 10000 samples
Epoch 1/250
11264/50000 [=====>........................] - ETA: 26:45 - loss: 2.3609 - accuracy: 0.1308

KeyboardInterrupt: 

In [11]:
# VGG753 with Batch Norm instead of LRN

print('Training VGG753 with seed', str(1000))
K.clear_session()
vgg753 = init_model('vgg753', seed=2)

def scheduler(epoch, lr):
    if (epoch == 100 or epoch == 150 or epoch == 200):
        return lr * 0.1
    return lr
    
lr_callback = LearningRateScheduler(scheduler)

a_history = vgg753.fit(
            x_train,
            y_train,
            batch_size=512,
            epochs=250,
            callbacks=[lr_callback],
            validation_data=(x_test, y_test),
            shuffle=True)

Training VGG753 with seed 1000
Train on 50000 samples, validate on 10000 samples
Epoch 1/250
  512/50000 [..............................] - ETA: 1:27

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-11-58ad6d8ae876>:21) ]] [Op:__inference_distributed_function_5271]

Function call stack:
distributed_function


In [5]:
from tensorflow.keras.models import load_model
model = load_model('Models/Primary/Version_1/all_cnn_c_0.h5')


In [7]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 96)        2688      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 96)        83040     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 96)        83040     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 192)       166080    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 192)       331968    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 192)         331968    
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 192)         3